# Language model in character/word level 

## Preprocessing of data

In [1]:
import numpy as np
import pandas as pd
import nltk, itertools, re
import itertools, operator
from collections import Counter

# data = open('data/reddit-comments-2015-08.csv', 'r').read()

df = pd.read_csv('data/reddit-comments-2015-08.csv')

# with open('data/reddit-comments-2015-08.csv', 'r') as f:
#     reader = 

df.head()

,body
0,I joined a new league this year and they have ...
1,"In your scenario, a person could just not run ..."
2,They don't get paid for how much time you spen...
3,"I dunno, back before the August update in an A..."
4,"No, but Toriyama sometimes would draw himself ..."


In [2]:

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()



In [3]:
# X_train = df.body.str.lower()
X_train = df.body.map(lambda x: clean_str(x))
X_train.head()

0    i joined a new league this year and they have ...
1    in your scenario , a person could just not run...
2    they do n't get paid for how much time you spe...
3    i dunno , back before the august update in an ...
4    no , but toriyama sometimes would draw himself...
Name: body, dtype: object

In [4]:
#if read from a plain text file
data = open('data/text.txt', 'r').read()
data


'I joined a new league this year and they have\n'

#### Char-level data processing

In [5]:
char_data = [clean_str(s.strip()) for s in df.body]
char_data = '.'.join(char_data)
char_data
# #Consider char-level
# char_list = "abcdefghijklmnopqrstuvwxyz0123456789 ,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"

chars = list(set(char_data))
chars=sorted(chars)
print(chars)
print("Data has %d characters in which %d are unique" %(len(char_data), len(chars)))


[' ', '!', "'", '(', ')', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', '\\', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Data has 7500269 characters in which 46 are unique


In [6]:
# create char-to-index, index-to-char lists
unknown_char = '§'
index_to_char = [unknown_char] +[x[0] for x in chars] 
char_to_index = {c:i for i, c in enumerate(index_to_char)}  
print(char_to_index, "\n", index_to_char)
 
#find a maximum length of a row in the X_train
#this will be used in the case "considering each row of data as a 'full' sentence
lenchar_of_row=X_train.map(len).max()
print(lenchar_of_row)

{'§': 0, ' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, ',': 6, '.': 7, '0': 8, '1': 9, '2': 10, '3': 11, '4': 12, '5': 13, '6': 14, '7': 15, '8': 16, '9': 17, '?': 18, '\\': 19, '`': 20, 'a': 21, 'b': 22, 'c': 23, 'd': 24, 'e': 25, 'f': 26, 'g': 27, 'h': 28, 'i': 29, 'j': 30, 'k': 31, 'l': 32, 'm': 33, 'n': 34, 'o': 35, 'p': 36, 'q': 37, 'r': 38, 's': 39, 't': 40, 'u': 41, 'v': 42, 'w': 43, 'x': 44, 'y': 45, 'z': 46} 
 ['§', ' ', '!', "'", '(', ')', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', '\\', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
9519


In [7]:
######Convert text into integer ######
# Method 1: consider chars of each sentence differently

ss = X_train.str.pad(lenchar_of_row, side='right', fillchar=unknown_char).\
        map(lambda st: [char_to_index[c] for c in st])
# lenchar_of_row=X_trailencharn_char.map(len).max()
# X_train_char = X_train_char.as_matrix()
# print(len(X_train_char[33]))

print(ss.head())
# ss = pd.Series(data = ss)
X_train_char = np.matrix(ss.tolist())
print(type(X_train_char), X_train_char.shape)

0    [29, 1, 30, 35, 29, 34, 25, 24, 1, 21, 1, 34, ...
1    [29, 34, 1, 45, 35, 41, 38, 1, 39, 23, 25, 34,...
2    [40, 28, 25, 45, 1, 24, 35, 1, 34, 3, 40, 1, 2...
3    [29, 1, 24, 41, 34, 34, 35, 1, 6, 1, 22, 21, 2...
4    [34, 35, 1, 6, 1, 22, 41, 40, 1, 40, 35, 38, 2...
Name: body, dtype: object
<class 'numpy.matrixlib.defmatrix.matrix'> (15000, 9519)


In [8]:
print(type(ss))

<class 'pandas.core.series.Series'>


In [9]:
#Just reference to see the speed of finding max
%timeit -n 100 df.body.str.len().max()
%timeit -n 100 df.body.map(lambda x: len(x)).max()
%timeit -n 100 df.body.map(len).max()


5.32 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.73 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.42 ms ± 96 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Word-level data processing

In [10]:
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

#Consider word-level
# word_data = ["%s %s %s" %(sentence_start_token, s, sentence_end_token) for s in df.body]
# word_data[0]

def getTokenizedSentences(lines, min_sent_characters=1):
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x) for x in lines])
    sentences = [clean_str(s) for s in sentences if "http" not in s and len(s) >= min_sent_characters]
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
    return sentences

sents = getTokenizedSentences(X_train)
print(sents[133])


SENTENCE_START gt please have adam sandler be steve then again , he already got the movie rights for many video game characters in pixels i would n't be surprised if the guy was able to score another big role relating to video games SENTENCE_END


In [11]:
vocabulary_size = 8000

def buildVocab(sentences):
    # Tokenize the sentences into words
    tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

    ####### If we want to order the words in alphabet #######
    # Count the word frequencies
    word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    print("Found %d unique words tokens." % len(word_freq.items()))

    # Get the most common words and build index_to_word and word_to_index vectors
    vocab = sorted(word_freq.items(), key=lambda x: (x[1], x[0]), reverse=True)[:vocabulary_size-2]
    print("Using vocabulary size %d." % vocabulary_size)
    print("The most frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[0][0], vocab[0][1]))
    print("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))
 
    sorted_vocab = sorted(vocab, key=operator.itemgetter(1))
    index_to_word = ["<MASK/>", unknown_token] + [x[0] for x in sorted_vocab]
    word_to_index = dict([(w, i) for i, w in enumerate(index_to_word)])
    
    
    print(word_to_index)
    return index_to_word, word_to_index
    
index_to_word, word_to_index = buildVocab(sents)

Found 46832 unique words tokens.
Using vocabulary size 8000.
The most frequent word in our vocabulary is 'the' and appeared 46857 times.
The least frequent word in our vocabulary is 'viewer' and appeared 8 times.
{'<MASK/>': 0, 'UNKNOWN_TOKEN': 1, 'zealand': 2, 'yoga': 3, 'yarn': 4, 'ww2': 5, 'wvw': 6, 'wotlk': 7, 'witness': 8, 'wires': 9, 'wipes': 10, 'williams': 11, 'whoops': 12, 'whale': 13, 'weighs': 14, 'weaver': 15, 'wax': 16, 'washed': 17, 'warranted': 18, 'warp': 19, 'warmth': 20, 'warlocks': 21, 'warehouse': 22, 'wandering': 23, 'wallets': 24, 'vw': 25, 'vpn': 26, 'voter': 27, 'vocals': 28, 'visually': 29, 'visits': 30, 'violations': 31, 'viewer': 32, 'zwei': 33, 'zelda': 34, 'yourselves': 35, 'yearly': 36, 'xanax': 37, 'writingprompts': 38, 'wrecked': 39, 'worms': 40, 'wonders': 41, 'wolfenstein': 42, 'witnessed': 43, 'wished': 44, 'willingness': 45, 'wilderness': 46, 'whiskey': 47, 'whim': 48, 'wednesday': 49, 'weddings': 50, 'wd': 51, 'warped': 52, 'warn': 53, 'vous': 54, '

In [12]:


####### In case use all sentences of each element in the df.body ######
# Replace all words not in our vocabulary with the unknown token
tokenized_sentences = []
for i, sent in enumerate(sents):
    temp = [word_to_index[w] if w in word_to_index else word_to_index[unknown_token] for w in sent.split()]
    tokenized_sentences.append(temp)

# #convert text to integers
# tokenized_sents = [[word_to_index[c] if c in word_to_index else word_to_index[unknown_token] \
#                                        for c in s] for s in sents.split()]
print(tokenized_sentences[3], sents[3])
X_train_word = tokenized_sentences
len(X_train_word)

[7992, 7987, 7921, 7983, 5863, 7989, 1, 7993] SENTENCE_START that does n't convince you \? SENTENCE_END


21651

In [17]:
# Convert training data's text to integer

####### In case considering each row in df.body is a sentence ######

lenword_of_row = X_train.str.split().str.len().max() 

print("maxlen(sentence)=",lenword_of_row)

def manipulateSentence(s, sequence_length, min_sent_characters=1):
    if ("http" not in s and len(s) >= min_sent_characters):
         s = clean_str(s)
    
    #Pad special words. Note: Adding <START/END> is not obligation here as 1 row is 1 sentence
    num_padding = sequence_length - len(s.split())
    if num_padding >0:
        temp = str([unknown_token] * num_padding)
        s = ' '.join([s, temp])

    # Append SENTENCE_START and SENTENCE_END
    s = " ".join([sentence_start_token, s, sentence_end_token])
    
    #convert word to integer
    s = [word_to_index[c] if c in word_to_index else word_to_index[unknown_token] \
                                for c in s.split()]
    if (len(s)!=sequence_length+2): print(len(s), sequence_length)
    return s

ss = X_train.map(lambda s: manipulateSentence(s, lenword_of_row))
print(ss.head())

lenword_of_row+=2
# ss = X_train.map(lambda x: [word_to_index[c] if c in word_to_index else word_to_index[unknown_token] \
#                                        for c in x.split()] )



maxlen(sentence)= 1936
0    [7992, 7996, 4556, 7995, 7852, 7255, 7970, 780...
1    [7992, 7985, 7961, 6677, 7998, 7995, 7782, 791...
2    [7992, 7973, 7974, 7983, 7948, 7269, 7984, 792...
3    [7992, 7996, 5549, 7998, 7877, 7851, 7999, 565...
4    [7992, 7934, 7998, 7980, 1, 7613, 7956, 6551, ...
Name: body, dtype: object


In [18]:
x_temp = ss.values #[[5,2],[1,2,3],[1]]
length = len(sorted(x_temp,key=len, reverse=True)[0])
X_train_word = np.array([xi+[None]*(length-len(xi)) for xi in x_temp])
X_train_word.shape

(15000, 1938)

## Char-level Model

In [19]:
hidden_size = 200 # nb of neurons in hidden layer
seq_length = 25 # number of steps to unroll the RNN for, and this is also nb of chars putting in the input of RNN
learning_rate = 1e-1

vocab_size = len(chars) + 1

#init parameters
W_hh = np.random.randn(hidden_size, hidden_size) #(H,H)
W_xh = np.random.randn(vocab_size, hidden_size) #(D,H)
W_hy = np.random.randn(hidden_size, vocab_size) #(H,M), M=D
b_h = np.zeros((1,hidden_size)) #(H,)
b_y = np.zeros((1,vocab_size)) #(M,)


## Layers

In [20]:
#Reference from @Karpathy: https://gist.github.com/karpathy/d4dee566867f8291f086
#
#                         [b_h]                                                [b_y]
#    w2v                    v                                (h_next)            v
#  x --> x_s -> [W_xh] -> [sum] -> h_raw -> [nonlinearity] -> h_s -> [W_hy] -> [sum] -> y_s -> [exp(y[k])/sum(exp(y))] -> p_s
#                           ^                                  |
#                           '----h_prev------[W_hh]------------'
#


### RNN Forward

In [41]:
def rnn_forward_single_step(x, h_prev, W_xh, W_hh, W_hy, b_h, b_y):
    """
    Run the forward pass for a single timestep of a vanilla RNN that uses a tanh
    activation function.

    The input data has dimension D, the hidden state has dimension H, and we use
    a minibatch size of N.

    Inputs:
    - x: Input data for this timestep, of shape (N, D).
    - h_prev: Hidden state from previous timestep, of shape (N, H)
    - W_xh: Weight matrix for input-to-hidden connections, of shape (D, H)
    - W_hh: Weight matrix for hidden-to-hidden connections, of shape (H, H)
    - W_hy: Weight matrix for hidden-to-output connections, of shape (H, M)
    - b_h: Biases of shape (H,)
    - b_y: Bias of shape (M, )

    Returns a tuple of:
    - h_next: Next hidden state, of shape (N, H)
    - y_s: output of this timestep (N, M)
    - cache: Tuple of values needed for the backward pass.
    """
    
    h_raw = np.dot(x, W_xh) + np.dot(h_prev, W_hh) + b_h  #(N,D)x(D,H) + (N,H)x(H,H) +(1,H) = (N,H)
    h_next = np.tanh(h_raw) #(N, H)
    y_s = np.dot(h_next, W_hy) + b_y #(N, H)x(H,M) +(1,M) =(N,M)
#     p_s = np.exp(y_s) / np.sum(np.exp(y_s), axis=0)
    
    cache = (x, h_prev, h_next, W_xh, W_hh, W_hy, b_h, b_y)
    return h_next, y_s, cache


def rnn_forward(x, h0, W_xh, W_hh, W_hy, b_h, b_y):
    """
    Run a vanilla RNN forward on an entire sequence of data. We assume an input
    sequence composed of T vectors (each vector represents a word/char), each of dimension D. 
    The RNN uses a hidden
    size of H, and we work over a minibatch containing N sequences. After running
    the RNN forward, we return the hidden states for all timesteps.
    Inputs:
    - x: Input data for the entire timeseries, of shape (N, T, D).
    - h0: Initial hidden state, of shape (N, H)
    - W_xh: Weight matrix for input-to-hidden connections, of shape (D, H)
    - W_hh: Weight matrix for hidden-to-hidden connections, of shape (H, H)
    - W_hy: Weight matrix for hidden-to-output connections, of shape (H, M)
    - b_h: Biases of shape (H,)
    - b_y: Bias of shape (M, )
    Returns a tuple of:
    - h: Hidden states for the entire timeseries, of shape (N, T, H)
    - y: Output of the entire timeseries, of shape (N, T, M)
    - cache: Values needed in the backward pass
    """
    N, T, D = x.shape
    H, M = W_hy.shape
    
    h = np.empty((N, T, H))
    cache = {}
    y_s = np.empty((N, T, M))
    
    print("N=%d, T=%d, D=%d, H=%d, M=%d" %(N, T,D,H,M))
    for i in range(T):
        if i==0: 
            h[:, i, :], y_s[:, i, :], cache[i] = rnn_forward_single_step(x[:,i,:], h0, W_xh, W_hh, W_hy, b_h, b_y)
        else: 
            h[:, i, :], y_s[:, i, :], cache[i] = rnn_forward_single_step(x[:,i,:], h[:, i-1, :], W_xh, W_hh, W_hy, b_h, b_y)
    
    return h, y_s, cache




### RNN Backward

In [42]:
#Reference from @Karpathy: https://gist.github.com/karpathy/d4dee566867f8291f086
#
#                         [b_h] (1,H)                                         [b_y] (1,M)
#    w2v                    v      (N,H)                     (N,H)   (H,M)       v
#  x --> x_s -> [W_xh] -> [sum] -> h_raw -> [nonlinearity] -> h_s -> [W_hy] -> [sum] -> y_s -> [exp(y[k])/sum(exp(y))] -> p_s
# (N,D)         (D,H)       ^                                  |                       (N,M)
#                           '----h_prev------[W_hh]------------'
#                                (N,H)       (H,H)

def rnn_backward_single_step (dh_next, dy, cache):
    """
    Backward pass for a single timestep of a vanilla RNN.
    Inputs:
    - dh_next: Gradient of loss with respect to next hidden state (N, H)
    - dy: of shape (N,M)
    - cache: Cache object from the forward pass
    Returns a tuple of:
    - dx: Gradients of input data, of shape (N, D)
    - dh_prev: Gradients of previous hidden state, of shape (N, H)
    - dWxh: Gradients of input-to-hidden weights, of shape (D, H)
    - dWhh: Gradients of hidden-to-hidden weights, of shape (H, H)
    - dWhy: Gradients of hidden-to-output weights, of shape (H, M)
    - dbh: Gradients of bias vector, of shape (H,)
    - dby: Gradients of bias vector, of shape (M,)
    
    """
    dx, dh_prev, dWxh, dWhh, dWhy, dbh, dby = None, None, None, None, None, None, None
    x, h_prev, h_next, W_xh, W_hh, W_hy, b_h, b_y = cache
    
    
    
    dby = np.sum(dy, axis=0) #(1,M)
    dWhy = np.dot(h_next.T, dy) #(H,N)x(N,M) = (H,M)
    dh = np.dot(dy, Why.T) + dh_next # backprop into h, (N,M)x(M,H)=(N,H)
    dh_raw = (1 - h_next ** 2) * dh # note: tanh(x)' = 1 - tanh^2(x), shape=(N,H)
    dbh = np.sum(dh_raw, axis=0) #(1,H)
    dWxh = np.dot(x.T, dh_raw) # (D,N)x(N,H) = (D,H)
    dWhh = np.dot(h_prev.T, dh_raw) # (H,N)x(N,H)=(H,H)
    dx = np.dot(dh_raw, W_xh.T) # (N,H)x(H,D) = (N,D)
    dh_prev = np.dot(dh_raw, W_hh.T) # (N,H)x(H,H) = (N,H)
    
    ########
#     dpre_actv = (1 - next_h ** 2) * dnext_h         # (N, H)
#     dx = dpre_actv.dot(Wx.T)
#     dprev_h = dpre_actv.dot(Wh.T) #(N,H)x(H,H)=(N,H)
#     dWx = x.T.dot(dpre_actv)
#     dWh = prev_h.T.dot(dpre_actv)
#     db = np.sum(dpre_actv, 0)
    
    
#     dWhy += np.dot(dy, hs[t].T)
#     dby += dy
#     dh = np.dot(Why.T, dy) + dhnext # backprop into h
#     dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
#     dbh += dhraw
#     dWxh += np.dot(dhraw, xs[t].T)
#     dWhh += np.dot(dhraw, hs[t-1].T)
#     dhnext = np.dot(Whh.T, dhraw)
    
    return dx, dh_prev, dWxh, dWhh, dWhy, dbh, dby


def rnn_backward(dh, dy, cache):
    """
    Compute the backward pass for a vanilla RNN over an entire sequence of data.
    Inputs:
    - dh: Upstream gradients of all hidden states, of shape (N, T, H)
    - dy: Upstream gradients of output, of shape (N, M)
    - cache
    Returns a tuple of:
    - dx: Gradient of inputs, of shape (N, T, D)
    - dh0: Gradient of initial hidden state, of shape (N, H)
    - dWx: Gradient of input-to-hidden weights, of shape (D, H)
    - dWh: Gradient of hidden-to-hidden weights, of shape (H, H)
    - dbh: Gradient of biases, of shape (H,)
    - dby: Gradient of biases, of shape (M,)
    """
    N, T, H =dh.shape
    N, M = dy.shape
    
    
    dx = np.empty((N,T,D))
    dWxh = np.zeros((D,H))
    dWhh = np.zeros((H,H))
    dWhy = np.zeros((H,M))
    dbh = np.zeros((1,H))
    dby = np.zeros((1,M))
    dh_prev = 0
    
    for i in reversed(range(T)):
        # Add the current timestep upstream gradient to previous calculated dh
        dh_next = dh_prev + dh[:,i,:]
        dx[:,i,:], dh_prev, dWxh_temp, dWhh_temp, dWhy_temp, dbh_temp, dby_temp = \
                rnn_backward_single_step(dh_next, dy[:,i,:], cache[i])
        dWxh += dWxh_temp
        dWhy += dWhy_temp
        dWhh += dWhh_temp
        dbh += dbh_temp
        dby += dby_temp
    
    dh0 = dh_prev
    return dx, dh0, dWxh, dWhh, dWhy, dbh, dby
    

### Affine forward/backward

In [43]:
## Affine forward
def affine_forward(x, W, b):
    """
    Inputs:
    x - Input data, of shape (N, D) #may be generally (N,d1,...d_k) 
    w - Weights, of shape (D, M)
    b - Biases, of shape (M,)

    Returns a tuple of:
    - out: output, of shape (N, M)
    - cache: (x, w, b)
    """
    
    out = np.dot(np.reshape(x,(N,-1)), W) + b #out = xW+b
    cache = (x, W, b)
    return out, cache

## Affine backward
def affine_backward(dout, cache):
    """
    Inputs:
    - dout: Upstream derivative, of shape (N, M)
    - cache: Tuple of:
      - x: Input data, of shape (N, D)
      - w: Weights, of shape (D, M)

    Returns a tuple of:
    - dx: Gradient with respect to x, of shape (N, D)
    - dw: Gradient with respect to w, of shape (D, M)
    - db: Gradient with respect to b, of shape (M,)
    """    
    x, W, b = cache
    db = np.sum(dout, axis=0)
    dx = dout.dot(W.T).reshape(x.shape)
    dw = x.reshape(x.shape[0], -1).T.dot(dout)
    

### Softmax

In [93]:
def temporal_softmax_loss (y_s, y, mask, verbose=False):
    """
    A temporal version of softmax loss for use in RNNs. We assume that we are
    making predictions over a vocabulary of size M for each timestep of a
    timeseries of length T, over a minibatch of size N. The input y_s gives SCORES
    for all vocabulary elements at all timesteps, and y gives the INDICES of the
    ground-truth element at each timestep. We use a cross-entropy loss at each
    timestep, summing the loss over all timesteps and averaging across the minibatch.
    
    As an additional complication, we may want to ignore the model output at some
    timesteps, since sequences of different length may have been combined into a
    minibatch and padded with NULL tokens. The optional mask argument tells us
    which elements should contribute to the loss.
    
    Inputs:
    - y_s: Input scores, of shape (N, T, M)
    - y: Ground-truth indices, of shape (N, T) where each element is in the range
         0 <= y[i, t] < M
    - mask: Boolean array of shape (N, T) where mask[i, t] tells whether or not
      the scores at y_s[i, t] should contribute to the loss.
    Returns a tuple of:
    - loss: Scalar giving loss
    - dy_s: Gradient of loss with respect to scores y_s, of shape (N, T, M)
    """
    
    N, T, M = y_s.shape

    ys_flat = y_s.reshape(N * T, M)
    y_flat = y.reshape(N * T, 1)
    mask_flat = mask.reshape(N * T, 1)

    print("mask, mask_flat shapes = ", mask.shape, mask_flat.shape)
    print("ys, ys_flat shapes = ", y_s.shape, ys_flat.shape)
    print("y, y_flat shapes = ", y.shape, y_flat.shape)
    
    
    probs = np.exp(ys_flat - np.max(ys_flat, axis=1, keepdims=True))
    probs /= np.sum(probs, axis=1, keepdims=True)
    print("-----calculating loss..... -------", mask_flat.shape, y_flat.shape)
    loss = -np.sum(mask_flat * np.log(probs[np.arange(N * T), y_flat])) / N
    dys_flat = probs.copy()
    dys_flat[np.arange(N * T), y_flat] -= 1
    dys_flat /= N
    print("-----calculating dx -------", dys_flat.shape, mask_flat.shape)
    dys_flat *= mask_flat[:, None]

    if verbose: print('dys_flat: ', dys_flat.shape)

    dy_s = dys_flat.reshape(N, T, M)

    return loss, dy_s
    

### Word embedding

In [94]:
def word_embedding_forward(x, W):
    """
    Forward pass for word embeddings. We operate on minibatches of size N where
    each sequence has length T. We assume a vocabulary of V words, assigning each
    to a vector of dimension D.
    Inputs:
    - x: Integer array of shape (N, T) giving indices of words. Each element idx
      of x muxt be in the range 0 <= idx < V.
    - W: Weight matrix of shape (V, D) giving word vectors for all words.
    Returns a tuple of:
    - out: Array of shape (N, T, D) giving word vectors for all input words.
    - cache: Values needed for the backward pass
    """
    out = W[x, :] 
    cache = (x, W)
    return out, cache


def word_embedding_backward(dout, cache):
    """
    Backward pass for word embeddings. We cannot back-propagate into the words
    since they are integers, so we only return gradient for the word embedding
    matrix.
    Inputs:
    - dout: Upstream gradients of shape (N, T, D)
    - cache: Values from the forward pass
    Returns:
    - dW: Gradient of word embedding matrix, of shape (V, D).
    """
    
    # x: (N, T)
    # W: (V, D)
    x, W = cache
    N, T, D = dout.shape

    dW = np.zeros_like(W)
    np.add.at(dW, x.reshape([-1]), dout.reshape([-1, D]))
    
    return dW



## Classifier

In [95]:
#Initialisation
#Here, some vars are repeated to memorize

hidden_size = 200 # H, nb of neurons in hidden layer
seq_length = 25 # T, nb of steps to unroll the RNN for <=> nb of chars putting in the input of RNN
data_size = len(df.body)


type = "char"
vocab_size = len(chars)+1 # D
wordvec_size = vocab_size # V, if 'char' then D=V as we don't use wordvec

if type=="word":
    data_size = X_train_word.shape[0] #len(sents)
    vocab_size = len(word_to_index)
    wordvec_size = 128 # output embedding vector length

#init word/char vector
W_embed = np.random.randn(vocab_size, wordvec_size)
W_embed /=100


#init RNN parameters
W_xh = np.random.randn(wordvec_size, hidden_size) #(V,H)
W_xh /= wordvec_size
W_hh = np.random.randn(hidden_size, hidden_size) #(H,H)
W_hh /= hidden_size
W_hy = np.random.randn(hidden_size, vocab_size) #(H,M)
W_hy /= hidden_size
b_h = np.zeros((1, hidden_size)) #(H,)
b_y = np.zeros((1, vocab_size)) #(M,)

h_prev = np.zeros((1, hidden_size))

### Training loss

## Solver

In [96]:
# parameters of training
update_rule = 'sgd'
optim_config={'learning_rate': 1e-2}
lr_decay = 1.0
data_size = df.body.shape[0]
batch_size = 2 # N
num_epochs = 10 # E
print_every = 100
verbose = True
loss_history = []

#### Update rules 

In [97]:

def sgd(w, dw, config=None):
    """
    Vanilla SGD
    config format:
    - learning_rate: Scalar learning rate.
    """
    if config is None: config = {}
    config.setdefault('learning_rate', 1e-2)

    w -= config['learning_rate'] * dw
    return w, config


def adam(x, dx, config=None):
    """
    Uses the Adam update rule, which incorporates moving averages of both the
    gradient and its square and a bias correction term.

    config format:
    - learning_rate: Scalar learning rate.
    - beta1: Decay rate for moving average of first moment of gradient.
    - beta2: Decay rate for moving average of second moment of gradient.
    - epsilon: Small scalar used for smoothing to avoid dividing by zero.
    - m: Moving average of gradient.
    - v: Moving average of squared gradient.
    - t: Iteration number.
    """
    if config is None: config = {}
    config.setdefault('learning_rate', 1e-3)
    config.setdefault('beta1', 0.9)
    config.setdefault('beta2', 0.999)
    config.setdefault('epsilon', 1e-8)
    config.setdefault('m', np.zeros_like(x))
    config.setdefault('v', np.zeros_like(x))
    config.setdefault('t', 0)

    next_x = None
    beta1, beta2, eps = config['beta1'], config['beta2'], config['epsilon']
    t, m, v = config['t'], config['m'], config['v']
    m = beta1 * m + (1 - beta1) * dx
    v = beta2 * v + (1 - beta2) * (dx * dx)
    t += 1
    alpha = config['learning_rate'] * np.sqrt(1 - beta2 ** t) / (1 - beta1 ** t)
    x -= alpha * (m / (np.sqrt(v) + eps))
    config['t'] = t
    config['m'] = m
    config['v'] = v
    next_x = x

    return next_x, config

In [98]:

def trainingLoss(text, img_fea=None, mode='train', type='char'):
    """
    Compute training-time loss.
    Inputs:
    - img_fea: Image features (used for captioning), of shape (N, D)
    - text: Ground-truth texts (e.g., captions); an integer array of shape (N, T) where
        each element is in the range 0 <= y[i, t] < M
    Returns:
        If training=True, then run a test-time forward pass of the model and return:
        - scores: Array of shape (N, M) giving classification scores, where
          scores[i, c] is the classification score for y[i] and the word/char with index c.

        Else, then run a training-time forward and backward pass and
        return a tuple of:
        - loss: Scalar value giving the loss
        - grads: Dictionary of gradients 
    """
    
    # Divide 'text' into two pieces 
#     text_in = text.map(lambda x: x[:-1])
#     text_out = text.map(lambda x: x[1:])
    text_in = text[:, :-1]
    text_out = text[:, 1:]
    print(text_in.shape, text_out.shape) # (2, 9518) = N, T
    
    if type=='word':  mask = (text_out != word_to_index[unknown_token])
    else: mask = (text_out != char_to_index[unknown_char])
    
    print(mask, mask.shape) #(2, 9518) = N, T
    ##### FORWARD STEPS ######
    embedded_text, cache_word_embedding = None, None
    
    # Embed the input word captions.
#     if type=='word':
    embedded_text, cache_word_embedding = word_embedding_forward(text_in, W_embed)
#     else: embedded_text = text_in
    
    print(embedded_text.shape) # (N, T, D) = (2, 9518, 47)

    #RNN forward
    h, y_s, cache_rnn = rnn_forward(embedded_text, h_prev, W_xh, W_hh, W_hy, b_h, b_y)
    
    if mode=='test':
        return y_s
    
    loss, grads = 0.0, {}

    #Softmax
    loss, dy_s = temporal_softmax_loss(y_s, text_out, mask)
    
    ##### BACKWARD STEPS ######
    
    #Backprop dy_s to get gradients
    dx, dh0, dWxh, dWhh, dWhy, dbh, dby = rnn_backward(dy_s, cache_rnn)
    grads['Wxh'], grads['Whh'], grads['Why'], grads['bh'], grads['by'] = dbh, dWxh, dWhh, dbh, dby

    # Backprop dx to get gradient for word embedding weights.
#     if (type=='word'):
    dW_embed = word_embedding_backward(dx, cache_word_embedding)
    grads['W_embed'] = dW_embed
    
    return loss, grads
    

In [99]:
# def sample_minibatch(data, batch_size=2, datatype='train'):
#     mask = np.random.choice(data., )

def executeStep():
    """
    Execute one iteration of training
    """
    #sample data
    N = X_train.shape[0]
    batch_mask = np.random.choice(N, batch_size)
    

    #convert text into interger arrays
    if (type=='char'): 
        X_batch = X_train_char[batch_mask]
    else: X_batch = X_train_word[batch_mask]
    
    
    #compute loss for this batch
    loss, grads = trainingLoss(X_batch)

def train():
    """
    Train the model
    """
    num_train = data_size 
    iterations_per_epoch = max (num_train//batch_size, 1)
    num_iterations = num_epochs * iterations_per_epoch
    
    for t in range(num_iterations):
            #Execute one step of training
            executeStep()

            # print loss for debug 
#             if verbose and t % print_every == 0:
#                 print('(Iteration %d / %d) loss: %f' % (
#                        t + 1, num_iterations, loss_history[-1]))

            # Increment the epoch counter and decay the learning rate.
            epoch_end = (t + 1) % iterations_per_epoch == 0
            if epoch_end:
                self.epoch += 1
                for k in self.optim_configs:
                    optim_configs[k]['learning_rate'] *= lr_decay

            # Check train and val accuracy on the first iteration, the last
            # iteration, and at the end of each epoch.
            # TODO: Implement some logic to check Bleu on validation set periodically

        # At the end of training swap the best params into the model
        # self.model.params = self.best_params
        
        
        
        
#     n, p = 0, 0
#     mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
#     mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
#     smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
#     while True:
#         # prepare inputs (we're sweeping from left to right in steps seq_length long)
#         if p+seq_length+1 >= len(data) or n == 0: 
#             hprev = np.zeros((hidden_size,1)) # reset RNN memory
#             p = 0 # go from start of data
#         inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
#         targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

#           # sample from the model now and then
#           if n % 100 == 0:
#             sample_ix = sample(hprev, inputs[0], 200)
#             txt = ''.join(ix_to_char[ix] for ix in sample_ix)
#             print '----\n %s \n----' % (txt, )

#           # forward seq_length characters through the net and fetch gradient
#           loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
#           smooth_loss = smooth_loss * 0.999 + loss * 0.001
#           if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress

#           # perform parameter update with Adagrad
#           for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
#                                         [dWxh, dWhh, dWhy, dbh, dby], 
#                                         [mWxh, mWhh, mWhy, mbh, mby]):
#             mem += dparam * dparam
#             param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

#           p += seq_length # move data pointer
#         n += 1 # iteration counter 

train()

(2, 9518) (2, 9518)
[[ True  True  True ..., False False False]
 [ True  True  True ..., False False False]] (2, 9518)
(2, 9518, 47)
N=2, T=9518, D=47, H=200, M=47
mask, mask_flat shapes =  (2, 9518) (19036, 1)
ys, ys_flat shapes =  (2, 9518, 47) (19036, 47)
y, y_flat shapes =  (2, 9518) (19036, 1)
-----calculating loss..... ------- (19036, 1) (19036, 1)


ValueError: shapes (19036,1) and (19036,19036) not aligned: 1 (dim 1) != 19036 (dim 0)